In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('vinho.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is good
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0


In [4]:
def MinMaxScale(x, min, max):
    return (x - min) / (max - min)

In [5]:
mins = df.min()
maxs = df.max()

for column in df:
    df[column] = MinMaxScale(df[column], mins[column], maxs[column])

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is good
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.0
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.0
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.0
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,1.0
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.0


In [6]:
target = 'is good'
features = df.columns.to_list()
features.remove(target)

from sklearn.model_selection import train_test_split

def distance(x, y):
    return np.sqrt(sum((x-y)**2))

X_train, X_val, y_train, y_val = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

class KNearestNeighbors:
    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_val):
        # Lista para resultado
        y_pred = []
        # Para cada amostra de vale, vamos calcular sua distância até os dados de treino
        X_val_arr = X_val.to_numpy()
        X_train_arr = self.X_train.to_numpy()
        for row_val in X_val_arr:
            # Array para guardar [index_train, distancia]
            distances = []
            # Vamos percorrer os dados de treino colocando as distancias em nossa matriz
            for index_train, row_train in enumerate(X_train_arr):
                distances.append([index_train, distance(row_val, row_train)])
            # Vamos ordenar o array com base nas distancias (np.argsort retorna somente os indices)
            distances = sorted(distances, key=lambda x: x[1])
            # Pegando os k primeiros vizinhos
            nearestNeighbors = distances[0:self.k]
            # Ignorando agora as distancias
            nearestNeighbors = np.array(nearestNeighbors)[:,0]
            # Para cada vizinho, vamos analisar nos dados de treino o target
            result = []
            for neighbor in nearestNeighbors:
                result.append(y_train.to_numpy()[int(neighbor)])
            if np.array(result).sum() > len(result) / 2:
                y_pred.append(1)
            else:
                y_pred.append(0)

        return y_pred

    def accuracy(self, y_val, y_pred):
        total = 0
        for val, pred in zip(y_val, y_pred):
            if (val == pred):
                total += 1
        return total / len(y_val)

model = KNearestNeighbors(k=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
model.accuracy(y_pred, y_val)

0.684375

In [7]:
ks = [3, 5, 7, 9]
for k in ks:
  model = KNearestNeighbors(k=k)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  print(model.accuracy(y_pred, y_val))

0.71875
0.684375
0.709375
0.7
